In [ ]:
# PyTorch 라이브러리 import
import torch
import torch.nn as nn
from torchvision import transforms, datasets


# CIFAR-10 training set 및 test set 불러오기
train_dataset = datasets.CIFAR10(root = '.', train = True, download = True,
                                transform = transforms.ToTensor())
test_dataset = datasets.CIFAR10(root = '.', train = False, download = True,
                                transform = transforms.ToTensor())


class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    torch.manual_seed(2025)
    # MLP 구성
    self.affine1 = nn.Linear(3*32*32, 1024)
    self.bn1 = nn.BatchNorm1d(1024)
    self.activation1 = nn.GELU()
    self.drop1 = nn.Dropout(0.3)

    self.affine2 = nn.Linear(1024, 512)
    self.bn2 = nn.BatchNorm1d(512)
    self.activation2 = nn.GELU()
    self.drop2 = nn.Dropout(0.3)

    self.affine3 = nn.Linear(512, 256)
    self.bn3 = nn.BatchNorm1d(256)
    self.activation3 = nn.GELU()
    self.drop3 = nn.Dropout(0.2)

    self.affine4 = nn.Linear(256, 128)
    self.bn4 = nn.BatchNorm1d(128)
    self.activation4 = nn.GELU()
    self.drop4 = nn.Dropout(0.2)

    self.affine5 = nn.Linear(128, 10)

  def forward(self, x):
    x = x.view(-1, 32*32*3) # Flatten

    x = self.affine1(x)
    x = self.bn1(x)
    x = self.activation1(x)
    x = self.drop1(x)

    x = self.affine2(x)
    x = self.bn2(x)
    x = self.activation2(x)
    x = self.drop2(x)

    x = self.affine3(x)
    x = self.bn3(x)
    x = self.activation3(x)
    x = self.drop3(x)

    x = self.affine4(x)
    x = self.bn4(x)
    x = self.activation4(x)
    x = self.drop4(x)

    x = self.affine5(x)
    return x

model = MLP().cuda()

# 학습 수행
batchsize = 64
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batchsize, shuffle=True)
learning_rate = 0.001
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
criterion = nn.CrossEntropyLoss()
num_epochs = 100
iter = 0
model.train()
for epoch in range(num_epochs):
  for image, label in train_loader:
    image = image.cuda()
    label = label.cuda()

    output = model(image)
    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    iter += 1
print(f'epochs = {num_epochs}, batch_size = {batchsize}, iters = {iter} 수행완료!')

# 모델 평가

batchsize = 64
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batchsize, shuffle=True)

num_correct = 0

model.eval()

with torch.no_grad():
  for image, label in test_loader:
    image = image.cuda()
    label = label.cuda()
    output = model(image)
    pred = output.argmax(dim=1)
    num_correct += (pred == label).sum()

print(f'Accuracy : {num_correct / len(test_dataset) * 100:.2f} %')

100%|██████████| 170M/170M [00:15<00:00, 10.8MB/s]


epochs = 100, batch_size = 64, iters = 78200 수행완료!
Accuracy : 58.79 %
